#Import Lib

In [0]:
# [Language detection]
!pip install langdetect 

# [for PythaiNLP Library]
!pip install pythainlp

# [Syllble Count]
!pip install https://github.com/wannaphongcom/thai-syllables-cut/archive/master.zip
!pip install ssg

     |████████████████████████████████| 981 kB 14.3 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=3774cd5045bbbe949be275361fb2b7ae90029ff030d1af92e03655e26483e495
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-63ca01ce-b404-4529-b3ea-749e3b779727/bin/python -m pip install --upgrade pip' command.
     |█████▊                          | 2.4 MB 21.3 MB/s et

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 13.4 MB 3.4 kB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-63ca01ce-b404-4529-b3ea-749e3b779727/bin/python -m pip install --upgrade pip' command.
     \ 28 kB 6.6 MB/s
     |████████████████████████████████| 840 kB 7.8 kB/s 
  Created wheel for pythaisyllables: filename=pyt

In [0]:
!pip install langid

!pip install pycld2

!pip install fasttext-langdetect

!pip install transformers

     |████████████████████████████████| 1.9 MB 20.0 MB/s 
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941187 sha256=e35199ef421085646c35095cef8cd12ef9301802f5cd797ad03431b20283921d
  Stored in directory: /root/.cache/pip/wheels/93/95/a9/c292c9dd8cadb8f2359f1670ff198a40d47167b0be3236e1c8
Successfully built langid
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-63ca01ce-b404-4529-b3ea-749e3b779727/bin/python -m pip install --upgrade pip' command.
     |█▏                              | 1.5 M

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 3.8 MB 94.0 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-63ca01ce-b404-4529-b3ea-749e3b779727/bin/python -m pip install --upgrade pip' command.


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score



# [Language detection]
from langdetect import detect

# PythaiNLP Library
import pythainlp

# Word tokenize
from pythainlp.tokenize import word_tokenize

# [Syllbels count]
from pythainlp import subword_tokenize

# [Soundex]
from pythainlp.soundex import lk82



# # [Word to Pronunciate]
from pythainlp.transliterate import pronunciate

# [Spell check]
from pythainlp.spell import NorvigSpellChecker

# [Normalize]
from pythainlp.util import normalize

from pythainlp.tag import NER


import langid

import pycld2 as cld2


from ftlangdetect import detect


#Get Data

In [0]:
load_data = spark.sql(query)
# select some column in load_data
select_column = load_data.select('firstname', 'lastname').distinct() 

In [0]:
# Convert pyspark sql to pandas dataframe
data = select_column.toPandas()

In [0]:
name = data.copy()
name.display()

## Defind Function split word

In [0]:
# Split and Cleaning the name [preprocess step]
def wordSplit(name):
  text = name['firstname']
  text = wordClean(text)
  text_split = text.split(' ')
  text_split = [word for word in text_split if word.strip()]
#   text_cleaning = [word for word in text_split if word not in ('นาย','นางสาว','น.ส.','ว่าที่', 'ร.ต.', 'ด.ญ','.','-')]
#   print(text_cleaning)
  
  if len(text_split) >= 1:
    name['fname'] = text_split[0]
    if len(text_split) >= 2:
      name['lname'] = "".join(word for word in text_split[1:])
    else:
        name['lname'] = name['lastname']
  else:
    name['fname'] = 'null'
    name['lname'] = 'null'
  
  return name

In [0]:
def wordClean(word):
  word = word.replace('นาย', '')
  word = word.replace('นางสาว', '')
  word = word.replace('นาง', '')
  word = word.replace('น.ส.', '')
  word = word.replace('นส.', '')
  word = word.replace('ร.ต.', '')
  word = word.replace('ด.ญ.', '')
  word = word.replace('ด.ช.', '')
  word = word.replace('คุณ', '')
  word = word.replace('-', '')
  word = word.replace('.', '')
  return word

In [0]:
# name.apply(wordSplit, axis=1)
name_ready = name.apply(wordSplit, axis=1)

In [0]:
name_ready

##1. Identify Language Name
- Used lib = https://pypi.org/project/langdetect/
- List meaning result = https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes

### LangeDetect

In [0]:
detect('Tanapong')

Out[11]: {'lang': 'en', 'score': 0.6618552207946777}

In [0]:
def langDetect(word):
  try:
    language = detect(word)
    #print(language)
  except:
    # When function found number or other that can not detect the language.
    language = 'Failed'
    
  return language

In [0]:
name_ready['result'] = name_ready['fname'].apply(langDetect)
name_ready[['LangeDetect', 'LangeDetectScore']] = name_ready['result'].apply(pd.Series)
name_ready = name_ready.drop(columns=['result'])

In [0]:
name_ready.display()

### LangeId

In [0]:
 langid.classify("มานพ")

Out[15]: ('th', -12.3280189037323)

In [0]:
def langId(word):
  try:
    language = langid.classify(word)
    #language_score = langid.classify(word)[1]
    #print(language)
  except:
    # When function found number or other that can not detect the language.
    language = 'Failed'
    
  return language

In [0]:
name_ready['result'] = name_ready['fname'].apply(langId)
name_ready[['LangID', 'LangIDScore']] = name_ready['result'].apply(pd.Series)
name_ready = name_ready.drop(columns=['result'])

In [0]:
name_ready.display()

In [0]:
#pd.crosstab(name_ready['LangID'],columns='cnt')

### Comparer Langdetect VS langeid

In [0]:
from langdetect import detect
import langid

text = "จักรพัน"

# Using langdetect
langdetect_lang = detect(text)

# Using langid
langid_result = langid.classify(text)
langid_lang, langid_confidence = langid_result

# Convert confidence scores to percentages
langdetect_percentage = 1.0  # Langdetect doesn't provide explicit percentages, so assume 100%
#langid_percentage = langid_confidence * 100
langid_percentage = min(langid_confidence * 100, 100)

print("Language Detection Results (with Percentage Confidence):")
print(f"langdetect detected: {langdetect_lang} with {langdetect_percentage:.2f}% confidence")
print(f"langid detected: {langid_lang} with {langid_percentage:.2f}% confidence")

Language Detection Results (with Percentage Confidence):
langdetect detected: th with 1.00% confidence
langid detected: th with -3807.87% confidence


## 2. Model For Detect Used Pronound 
- model สำหรับ การออกเสียง

In [0]:
def wordPronunciate(word):
  word_pronunciate = pronunciate(word, engine="w2p")
  return word_pronunciate

In [0]:
wordPronunciate('ณัฐวุฒิ')

Corpus: thai_w2p
- Downloading: thai_w2p 0.2


  0%|          | 0/9772474 [00:00<?, ?it/s]

Out[23]: 'นัด-ถะ-วุด'

In [0]:
name_ready['fname_phonetic'] = name_ready['fname'].apply(wordPronunciate)
name_ready['lname_phonetic'] = name_ready['lname'].apply(wordPronunciate)

In [0]:
name_ready

## 2. Used Soundex

In [0]:
def getSoundex(word):
  soundex = lk82(word)
  return soundex

##3.Spell Check

In [0]:
checker = NorvigSpellChecker()
def wordCheck(word):
  try:
    w_normalize = normalize(word)
    w_tokenize = subword_tokenize(w_normalize, engine="ssg") # word_tokenize(w_normalize, engine="newmm")

    score = 0
    count = len(w_tokenize)
    result_freq = []
    for i in w_tokenize:
      freq = checker.freq(i)
      result = {'word':i, 'freq':freq}
      result_freq.append(result)
      # print(word, freq)
      if freq == 0:
        score += 0
      else:
        score += 1

    # Avg Score
    if score==0 or count == 0:
      avg_score = 0
    else:
      avg_score = score/count

  #   print('- -')
  #   print("Word orgin:", word, "|--> Word normalize:", w_normalize, "|--> Word tokenize:", w_tokenize)
  #   print("Result:", result_freq)
  #   print("Score:", score, "| Count:", count ,"| Avg score:", avg_score)

    return avg_score
  
  except:
    avg_score = 0
    return avg_score

In [0]:
name_ready['fname_score'] = name_ready['fname'].apply(wordCheck)
name_ready['lname_score'] = name_ready['lname'].apply(wordCheck)

In [0]:
name_ready.display()

##4. Name Entiry
- Model detect ว่าเป็นคนหรือไม่

In [0]:
def nameEntity(word):
  try:
    ner = NER("thainer")
    name_entity = ner.tag(word,pos=True, tag=False)
    #print(name_entity)
    ner_tag = name_entity[0][2]
#     if ner_tag == 'B-PERSON':
#       ner_tag = 'PERSON'
    #print(ner_tag)
    return ner_tag
  except:
    ner_tag = 'Failed'
    return ner_tag

In [0]:
nameEntity('คุณต้า ธนาพงษ์')

Corpus: thainer-1.4
- Downloading: thainer-1.4 1.4


  0%|          | 0/1872468 [00:00<?, ?it/s]

Out[31]: 'B-PERSON'

In [0]:
name_ready['fname_entity'] = name_ready['fname'].apply(nameEntity)
name_ready['lname_entity'] = name_ready['lname'].apply(nameEntity)

In [0]:
name_ready.display()

##5. SpellCheck

In [0]:
def nameChecking(word):
  # Spell Check
  checker = NorvigSpellChecker()
  def wordCheck(word):
    try:
      w_normalize = normalize(word)
      w_tokenize = subword_tokenize(w_normalize, engine="ssg") 

      score = 0
      count = len(w_tokenize)
      result_freq = []
      for i in w_tokenize:
        freq = checker.freq(i)
        result = {'word':i, 'freq':freq}
        result_freq.append(result)
        # print(word, freq)
        if freq == 0:
          score += 0
        else:
          score += 1

      # Avg Score
      if score==0 or count == 0:
        avg_score = 0
      else:
        avg_score = score/count

      print('- -')
      print("Word orgin:", word, "|--> Word normalize:", w_normalize, "|--> Word tokenize:", w_tokenize)
      print("Result:", result_freq)
      print("Score:", score, "| Count:", count ,"| Avg score:", avg_score)

      return avg_score

    except:
      avg_score = 0
      return avg_score
 
  def wordPronunciate(word):
    word_pronunciate = pronunciate(word, engine="w2p")
    print("- "*30)
    print(word_pronunciate)
    return word_pronunciate

      # Name Entity
  def nameEntity(word):
    try:
      ner = NER("thainer")
      name_entity = ner.tag(word,pos=True, tag=False)
      #print(name_entity)
      ner_tag = name_entity[0][2]
  #     if ner_tag == 'B-PERSON':
  #       ner_tag = 'PERSON'
      print("- "*30)
      print(ner_tag)
      return ner_tag
    except:
      ner_tag = 'Failed'
      print("- "*30)
      print(ner_tag)
      return ner_tag

  wordCheck2 = wordCheck(word)
  wordPronunciate2 =  wordPronunciate(word)
  nameEntity2 = nameEntity(word) 

In [0]:
nameChecking('้้ห')

- -
Word orgin: ้้ห |--> Word normalize: ห |--> Word tokenize: ['ห']
Result: [{'word': 'ห', 'freq': 0}]
Score: 0 | Count: 1 | Avg score: 0
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
บ้า-หฺร้อ
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
O


##6. Tranformer
- https://huggingface.co/pythainlp/thainer-corpus-v2-base-model?text=%E0%B8%89%E0%B8%B1%E0%B8%99%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD+%E0%B8%99%E0%B8%B2%E0%B8%87%E0%B8%AA%E0%B8%B2%E0%B8%A7%E0%B8%A1%E0%B8%B0%E0%B8%A5%E0%B8%B4%E0%B8%A7%E0%B8%B2+%E0%B8%9A%E0%B8%B8%E0%B8%8D%E0%B8%AA%E0%B8%A3%E0%B8%B0%E0%B8%94%E0%B8%B5+%E0%B8%AD%E0%B8%B2%E0%B8%A8%E0%B8%B1%E0%B8%A2%E0%B8%AD%E0%B8%A2%E0%B8%B9%E0%B9%88%E0%B8%97%E0%B8%B5%E0%B9%88%E0%B8%AD%E0%B8%B3%E0%B9%80%E0%B8%A0%E0%B8%AD%E0%B8%99%E0%B8%B2%E0%B8%87%E0%B8%A3%E0%B8%AD%E0%B8%87+%E0%B8%88%E0%B8%B1%E0%B8%87%E0%B8%AB%E0%B8%A7%E0%B8%B1%E0%B8%94%E0%B8%9A%E0%B8%B8%E0%B8%A3%E0%B8%B5%E0%B8%A3%E0%B8%B1%E0%B8%A1%E0%B8%A2%E0%B9%8C+%E0%B8%AD%E0%B8%B2%E0%B8%A2%E0%B8%B8+23+%E0%B8%9B%E0%B8%B5+%E0%B9%80%E0%B8%9E%E0%B8%B4%E0%B9%88%E0%B8%87%E0%B9%80%E0%B8%A3%E0%B8%B5%E0%B8%A2%E0%B8%99%E0%B8%88%E0%B8%9A%E0%B8%88%E0%B8%B2%E0%B8%81+%E0%B8%A1%E0%B8%AB%E0%B8%B2%E0%B8%A7%E0%B8%B4%E0%B8%97%E0%B8%A2%E0%B8%B2%E0%B8%A5%E0%B8%B1%E0%B8%A2%E0%B8%82%E0%B8%AD%E0%B8%99%E0%B9%81%E0%B8%81%E0%B9%88%E0%B8%99+%E0%B9%81%E0%B8%A5%E0%B8%B0%E0%B8%99%E0%B8%B5%E0%B9%88%E0%B8%84%E0%B8%B7%E0%B8%AD%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5%E0%B8%9B%E0%B8%A5%E0%B8%AD%E0%B8%A1+%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B8%84%E0%B8%99%E0%B9%84%E0%B8%A1%E0%B9%88%E0%B8%A1%E0%B8%B5%E0%B8%AD%E0%B8%A2%E0%B8%B9%E0%B9%88%E0%B8%88%E0%B8%A3%E0%B8%B4%E0%B8%87

In [0]:
import torch

In [0]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("pythainlp/thainer-corpus-v2-base-model")

tokenizer_config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

In [0]:
model = AutoModelForTokenClassification.from_pretrained("pythainlp/thainer-corpus-v2-base-model")

config.json:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/419M [00:00<?, ?B/s]

In [0]:
from pythainlp.tokenize import word_tokenize

In [0]:
name_ready['FullName'] = ' '+ name_ready['fname'] + name_ready['lname']

In [0]:
def thaiNer2(word):
  try: 
    cut = word_tokenize(word.replace(" ", "<_>"))
    inputs = tokenizer(cut,is_split_into_words=True,return_tensors="pt")

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]

    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

    def fix_span_error(words,ner):
        _ner = []
        _ner = ner
        _new_tag = []
        for i,j in zip(words,_ner):
            i=tokenizer.decode(i)
  #           print(i,j)
            if i.isspace() and j.startswith("B-"):
                j="O"
            if i=='' or i=='<s>' or i=='</s>':
                continue
            if i=="<_>":
                i=" "
  #           print("-"*5)
  #           print(i,j)
            _new_tag.append((i,j))
        return _new_tag

    ner_tag=fix_span_error(inputs['input_ids'][0],predicted_token_class)
  #   print(ner_tag)

    #Person scoring
    score = 0
    count = len(ner_tag)
    for i in ner_tag:
      if i[1].endswith("PERSON"):
        score += 1

  #   print("Sum Score:", score, " | ", "Count:", count)
  #   print("Avg Score:", score/count)
    avg_score = score/count
    return avg_score
  
  except:
    avg_score = 0
    
    return avg_score
  

In [0]:
import numpy as np
chunk_size = 1000  # Define the chunk size
num_chunks = int(np.ceil(len(name_ready) / chunk_size))

def process_chunk(chunk):
    return chunk['FullName'].apply(thaiNer2)

In [0]:
results = []
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(name_ready))
    chunk_result = process_chunk(name_ready[start_idx:end_idx])
    results.append(chunk_result)

name_ready['TranFormerScore'] = pd.concat(results)

In [0]:
display(name_ready)

In [0]:
name_ready['TranFormerScore'] = name_ready['FullName'].apply(thaiNer2)

In [0]:
display(name_ready)

#Train Model

In [0]:
import pandas as pd
df = pd.DataFrame({'age':[20,30,40,50],
              'sex':['male','female','female','male'],
              'region':['northwest','southwest','northeast','southeast'],
              'charges':[1000,1000,2000,2000]})
df.sex = (df.sex == 'female')
df.region = pd.Categorical(df.region)
df.region = df.region.cat.codes
#X = df.loc[:,['age','sex','region']]
#y = df.loc[:,['charges']]

In [0]:
df.region

Out[11]: 0    1
1    3
2    0
3    2
Name: region, dtype: int8

In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Create a DataFrame
data = pd.DataFrame({'category': ['a', 'b', 'c']})

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Fit the encoder to the data
encoder.fit(data['category'])

# Transform the data
encoded_data = encoder.transform(data['category'])

# Print the encoded data
print(encoded_data)

[0 1 2]


In [0]:
encoded_data

Out[5]: array([0, 1, 2])

#Save to Table

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType ,BooleanType ,FloatType

spark_name_ready = spark.createDataFrame(name_ready)
#spark_name_ready = spark_name_ready.withColumn("TranFormerScore",spark_name_ready["TranFormerScore"].cast(IntegerType()))
#spark_name_ready = spark_name_ready.withColumn("fname_score",spark_name_ready["fname_score"].cast(IntegerType()))
#spark_name_ready = spark_name_ready.withColumn("lname_score",spark_name_ready["lname_score"].cast(IntegerType()))

In [0]:
display(spark_name_ready)

In [0]:

path_delta = "/fullnamedetectscoring"
print("Save to Delta Table....")
spark_name_ready.write.format('delta').mode("overwrite").save(path_delta);

Save to Delta Table....
